In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import os
from src.utils import get_project_root_path
import src.config as config 

In [ ]:
ROOT = get_project_root_path()

In [ ]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)

In [ ]:
df_singalp6_preds = pd.read_parquet(ROOT + '/data/processed/df_SignalP6_predicted_labels.parquet.gzip')

In [ ]:
df_singalp6_preds.head()

In [ ]:
df_singalp6_preds_type_split = df_singalp6_preds[
    df_singalp6_preds['Type'] == 'SP'
]

In [ ]:
# pd.DataFrame(df_singalp6_preds_type_split[['SignalP6_predicted_labels', 'Label']].value_counts())

In [ ]:
# df_singalp6_preds_type_split[
#     df_singalp6_preds_type_split['SignalP6_predicted_labels'] == 'NNHHHHHHHHHHHHCCCCOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO'
# ]

In [ ]:
pd.DataFrame([x for x in ''.join(df_singalp6_preds_type_split['SignalP6_predicted_labels'].values.tolist())]).value_counts()

In [ ]:
transltation_dict_NO_SP = {
    'O': '',
    'H': '',
    'N': '',
    'C': '',
}

transltation_dict_SP = {
    'O': 'O',
    'H': 'S',
    'N': 'S',
    'C': 'S',

}

translation_dict_TAT = {
    'O': '',
    'H': '',
    'N': '',
    'C': '',
    'R': ''
}

translation_dict_LIPO = {
    'O': '',
    'H': '',
    'N': '',
    'C': '',
}